# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
import re

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support, make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponsetbl', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for colnm in y_test.columns:
    print('FEATURE: {}\n'.format(colnm))
    print(classification_report(y_test[colnm],y_pred_pd[colnm]))

FEATURE: related

             precision    recall  f1-score   support

          0       0.63      0.37      0.46      1553
          1       0.82      0.93      0.87      4949
          2       0.50      0.44      0.47        52

avg / total       0.77      0.79      0.77      6554

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5470
          1       0.83      0.41      0.55      1084

avg / total       0.88      0.89      0.87      6554

FEATURE: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

FEATURE: aid_related

             precision    recall  f1-score   support

          0       0.74      0.86      0.79      3881
          1       0.73      0.55      0.63      2673

avg / total       0.73      0.73      0.73      6554

FEATURE: medi

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
# Model results in dataframe format
def get_results(y_test, y_pred):
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for colnm in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[colnm], y_pred[:,num], average='weighted')
        report.at[num+1, 'Category'] = colnm
        report.at[num+1, 'f_score'] = f_score
        report.at[num+1, 'precision'] = precision
        report.at[num+1, 'recall'] = recall
        num += 1
    print('Aggregated f_score:', report['f_score'].mean())
    print('Aggregated precision:', report['precision'].mean())
    print('Aggregated recall:', report['recall'].mean())
    print('Accuracy:', np.mean(Y_test.values == y_pred))
    return report

In [47]:
report = get_results(y_test, y_pred)
report

Aggregated f_score: 0.930473497528
Aggregated precision: 0.933356817762
Aggregated recall: 0.943774794019


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,f_score,precision,recall
1,related,0.771454,0.772489,0.791883
2,request,0.872003,0.882947,0.888312
3,offer,0.993139,0.990866,0.995423
4,aid_related,0.725699,0.733044,0.73375
5,medical_help,0.893728,0.90047,0.922338
6,medical_products,0.931625,0.937387,0.95148
7,search_and_rescue,0.963655,0.964755,0.974367
8,security,0.977846,0.975446,0.984895
9,military,0.959665,0.963868,0.970552
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [56]:
parameters = {'vect__ngram_range': ((1, 1), (1, 2))
              ,'vect__max_df': (1.0,2.0)
              #,'clf__estimator__max_depth': [10,20,None],
              #,'clf__estimator__min_samples_leaf':[2, 5]
             }

cv = GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3
            #,n_jobs = 10
                 )

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [57]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
report2 = get_results(y_test, y_pred)
report2

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................
[CV] vect__max_df=2.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=2.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=2.0, vect__ngram_range=(1, 1) ......................
[CV] vect__max_df=2.0, vect__ngram_range=(1, 2) ......................


KeyboardInterrupt: 

In [ ]:
cv.best_estimator_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# AdaBoost classifier

def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

pipeline = new_model_pipeline()

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)
report3 = get_results(y_test, y_pred)
report3

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('model.pkl', 'wb'))

# load the model from disk
# loaded_model = pickle.load(open('model.pkl', 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.